# Import Required Library

In [ ]:
# Check Versions
import tensorflow as tf
import keras as k

print(tf.__version__)
print(k.__version__)

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import pandas as pd
from glob import glob 
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
train_directory = '../input/100-bird-species/birds/train'
test_directory = '../input/100-bird-species/birds/test'
valid_directory = '../input/100-bird-species/birds/valid'

In [ ]:
# Add PreProcessing layers to the front of VGG
vgg = VGG16(
    input_shape = IMAGE_SIZE + [3], 
    weights = 'imagenet',
    include_top = False
)

# dont train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of classes
folders = glob('../input/100-bird-species/birds/train/*')
len(folders)

In [ ]:
folders[:3]

In [ ]:
# custom layer(s)
x = Flatten()(vgg.output)
# x = Dense(1000, activation = 'relu')(x)
pred_layer = Dense(len(folders), activation = 'softmax')(x)

In [ ]:
# Create a model object
model = Model(inputs = vgg.input, outputs = pred_layer)

In [ ]:
# View Summary of structure of the Model
model.summary()

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer= 'adam',
    metrics = ['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)


In [ ]:
training_set = train_datagen.flow_from_directory(
    directory = train_directory,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

test_set = test_datagen.flow_from_directory(
    directory = test_directory,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
len(training_set), len(test_set)

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.

In [ ]:
# r = model.fit_generator(
#     training_set,
#     validation_data = test_set,
#     epochs = 5,
#     steps_per_epoch = len(training_set),
#     validation_steps = len(test_set)
# )

r = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 5,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

In [ ]:
model.save("birds_classification_vgg.h5")

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

loaded_model = load_model('birds_classification_vgg.h5', compile = False)

In [ ]:
labels = training_set.class_indices

In [ ]:
labels

In [ ]:
# Swap to map reverse
labels_reverse = {k:v for v,k in labels.items()}

In [ ]:
labels_reverse

In [ ]:
def output(location):
    img = load_img(location, target_size = (224, 224, 3))
    img = img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, [0])
    answer = loaded_model.predict(img)
    y_class = answer.argmax(axis = -1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels_reverse[y]
    return res

In [ ]:
img = '../input/100-bird-species/birds/valid/BALD EAGLE/2.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

In [ ]:
img = '../input/100-bird-species/birds/valid/ALBATROSS/4.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)